# Technical analysis
Roboquant comes out of box with support for 200 technical indicators. These are indicators ranging findicators such as ADX, MACD, RSI, Stochastic, Bollinger Bands to Candlestick pattern recognition. The underlying library that is used is the excellent TA-Lib. TA-Lib can be difficult to use, but roboquant wraps these indicators in a convenient layer, so they are very easy to use in strategies.

In [ ]:
%use roboquant
Welcome()

# Define the strategy
We start by defining the actual strategy. There are three essestial steps:

1) Create an instance of TAStrategy and define how many days it should collect data. If you choose this period too low, certain indicacators will fail. For example if you want a moving average over 11 days, the days should be at least 11. if you choose it too large, it will take longer before your strategy starts to generate signals.

2) Define what you want to use for generating BUY signals. This can be a combination of indicators and you can also include regular Kotlin code. The only restriction is that the last statement return a boolean; true if you want to BUY, false otherwise.

3) Define what you want to use for generating SELL signals. The same rules and conditions apply as for the BUY method.

Please note, that although not common best practice, the framework doesn't stop you from using bullish indicators to generate a SELL signal and bearish indicators to generate a BUY signal. 

In [ ]:
// Define a strategy with maximum of 15 days of history. It is important that these are enough days to calculate the various indicators.
val strategy = TAStrategy(15)

// We want to generate a BUY signal if we detect a candlestick pattern called Morningstar or 3WhiteSoldiers
strategy.buy {
    ta.cdlMorningStar(it) || ta.cdl3WhiteSoldiers(it)
}

// We want to generate a SELL signal if we detect a candlestick pattern called Two Crows and also see that the 5 day weighted average is below the 9 day close
strategy.sell {
    ta.cdl2Crows(it) && ta.ema(it.close, 5) < ta.ema(it.close, 9)
}


# Create and run the back test
Now we can use this strategy just like any other strategy. In this case we use the CSV files for US stocks that come included with this notebook (this is a very small set of stocks) 

In [ ]:
val roboquant =  Roboquant(strategy, AccountSummary())

val feed = CSVFeed("../data/US")
roboquant.run(feed)

# Evaluate the results
Now lets see what the outcome is. Spoiler alert, this strategy won't make you rich ;) 

In [ ]:
roboquant.broker.account.summary()

In [ ]:
roboquant.logger.summary()